In [ ]:
from tqdm.notebook import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta
from libWLS import WLS
from kalman_filter import kalman_smoothing
from tqdm.auto import tqdm
from scipy.spatial import distance
from time import time 
from pathlib import Path
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import glob as gl
import numpy as np
import scipy.optimize
import numpy as np  
import pandas as pd 
import os, csv, sys
import glob
import subprocess
import navpy

# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager


: 

In [ ]:
path = 'gnss_2024_06_08_13_33_sat.csv'
gnss = pd.read_csv(path, index_col=False)
filtered_gnss = gnss[gnss['ReceivedSvTimeNanos'] > 1e10]
measurement = filtered_gnss

: 

In [ ]:
# Lọc các vệ tinh thuộc GPS (ConstellationType == 1)
# Loc cac ve tinh co Multipath
measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'

measurement = measurement.loc[measurement['MultipathIndicator'] == 0]

measurement = measurement.loc[measurement['ReceivedSvTimeUncertaintyNanos'] < 100]

# Chỉ lấy các vệ tinh bắt GPS
measurement = measurement.loc[measurement['Constellation'] == 'G']
# Đảm bảo cột 'Svid' là chuỗi ký tự để có thể sử dụng hàm str.len()
measurement['Svid'] = measurement['Svid'].astype(str)
# Thêm số 0 vào trước 'Svid' nếu chiều dài của nó là 1
measurement.loc[measurement['Svid'].str.len() == 1, 'Svid'] = '0' + measurement['Svid']
# Tạo cột mới 'SvName' bằng cách nối 'Constellation' và 'Svid'
measurement['SvName'] = measurement['Constellation'] + measurement['Svid']

measurement = measurement.loc[measurement['CarrierFrequencyHz'] > 1275420030]

measurement

: 

In [ ]:
#chuyển giá trị thời gian về dạng date time
measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
#Chia measurement thành các epoch, mỗi epoch gồm các vệ tinh có cùng thời gian thu
measurement['Epoch'] = 0
measurement.loc[measurement['UnixTime'] - measurement['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
measurement['Epoch'] = measurement['Epoch'].cumsum()
measurement

: 

In [ ]:
# Khởi tạo biến để lưu trữ kết quả
filtered_measurement = pd.DataFrame()

# Duyệt qua từng epoch
for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    # Thêm kết quả vào DataFrame kết quả
    filtered_measurement = pd.concat([filtered_measurement, one_epoch])

# Reset index của DataFrame mới
filtered_measurement.reset_index(drop=True, inplace=True)
filtered_measurement = filtered_measurement.groupby('Epoch').apply(lambda x: x.sort_values(by='Svid')).reset_index(drop=True)
measurement = filtered_measurement
measurement.head(20)

: 

In [ ]:
#lấy dữ liệu ephemeris
ephemeris_data_directory = 'output'
manager = EphemerisManager(ephemeris_data_directory)

# while num_sat < 4:
#     one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    
#     if not one_epoch.empty:
#         timestamp = pd.to_datetime(one_epoch.iloc[0]['UnixTime'], unit='s')
#         one_epoch.set_index('SvName', inplace=True)
#         num_sat = len(one_epoch.index)
    
#     epoch += 1

measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
timestamps = pd.to_datetime(measurement['UnixTime'].unique(),unit='s')
ephemeris_data = pd.DataFrame()

for timestamp in timestamps:
    # Get satellite names for the current timestamp
    sats = measurement.loc[measurement['UnixTime'] == timestamp, 'SvName'].tolist()
    # Retrieve ephemeris data
    ephemeris = manager.get_ephemeris(timestamp, sats)
    # Append the retrieved ephemeris data to the list
    ephemeris_data = pd.concat([ephemeris_data,ephemeris])

#ephemeris = manager.get_ephemeris(timestamp, sats)
#timestamp
ephemeris_data.reset_index(drop=True, inplace=True)
ephemeris_data.head(20)

: 

In [ ]:
LIGHTSPEED = 2.99792458e8

def calculate_satellite_position(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    sv_position['SvClockBiasMeters'] = ephemeris['SVclockBias'] * LIGHTSPEED
    sv_position['SvClockDriftMetersPerSecond'] = ephemeris['SVclockDrift'] * LIGHTSPEED

    #calculate velocity
    Edot_k = n_0/(1-ephemeris['e']*np.cos(E_k))
    
    vdot_k = Edot_k * np.sqrt(1 - ephemeris['e']**2)/(1 - ephemeris['e']*np.cos(E_k))
    
    d_idotk = ephemeris['IDOT'] + 2 * vdot_k * (ephemeris['C_is'] * cos2Phi_k - ephemeris['C_ic']*sin2Phi_k)
    
    udot_k = vdot_k + 2 * vdot_k * (ephemeris['C_us']*cos2Phi_k - ephemeris['C_uc']*sin2Phi_k)
    
    rdot_k = ephemeris['e'] * A * Edot_k * np.sin(E_k) + 2 * vdot_k * (ephemeris['C_rs'] * cos2Phi_k - ephemeris['C_rc'] * sin2Phi_k)
    
    Omegadot_k = ephemeris['OmegaDot'] - OmegaDot_e
    
    x_plus_dot_k = rdot_k * np.cos(u_k) - r_k * udot_k*np.sin(u_k)
    y_plus_dot_k = rdot_k * np.sin(u_k) + r_k * udot_k*np.cos(u_k)
    
    sv_position['v_x'] = -x_k_prime * Omegadot_k * np.sin(Omega_k) + x_plus_dot_k * np.cos(Omega_k) - y_plus_dot_k * np.sin(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.cos(Omega_k) * np.cos(i_k) - d_idotk * np.sin(Omega_k)*np.sin(i_k))
    
    sv_position['v_y'] = x_k_prime * Omegadot_k * np.cos(Omega_k) + x_plus_dot_k * np.sin(Omega_k) + y_plus_dot_k * np.cos(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.sin(Omega_k) * np.cos(i_k) + d_idotk * np.cos(Omega_k)*np.sin(i_k))
    
    sv_position['v_z'] = y_plus_dot_k * np.sin(i_k) + y_k_prime * d_idotk * np.cos(i_k)

    return sv_position

# Run the function and check out the results:
measurement['ReceivedSvTimeNanos'] = pd.to_numeric(measurement['ReceivedSvTimeNanos'])
measurement['TimeOffsetNanos'] = pd.to_numeric(measurement['TimeOffsetNanos'])
sv_position = calculate_satellite_position(ephemeris_data, (measurement['ReceivedSvTimeNanos'] + measurement['TimeOffsetNanos'])*1e-9)
#xs = sv_position[['x_k', 'y_k', 'z_k']].to_numpy()
measurement['SvPositionXEcefMeters'] = sv_position['x_k']
measurement['SvPositionYEcefMeters'] = sv_position['y_k']
measurement['SvPositionZEcefMeters'] = sv_position['z_k']
measurement['SvVelocityXEcefMetersPerSecond'] = sv_position['v_x']
measurement['SvVelocityYEcefMetersPerSecond'] = sv_position['v_y']
measurement['SvVelocityZEcefMetersPerSecond'] = sv_position['v_z']
measurement['SvClockBiasMeters'] = sv_position['SvClockBiasMeters']
measurement['SvClockDriftMetersPerSecond'] = sv_position['SvClockDriftMetersPerSecond']
measurement.head(20)

: 

In [ ]:
measurement.to_csv('gnss_sat.csv')

: 

In [ ]:
epoch = 0
num_sat = 0

while num_sat < 4:
    one_epoch = measurement.loc[measurement['Epoch'] == epoch]
    if not one_epoch.empty:
        one_epoch.set_index('SvName', inplace=True)
        num_sat = len(one_epoch.index)
    epoch += 1
one_epoch

: 

In [ ]:
# Bắt đầu từ đây là đoạn mã xử lý dữ liệu đã tìm được
wls = WLS()
ecef_list = []
utcTimeMillis_list = []  # Danh sách để lưu giữ utcTimeMillis tương ứng

for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[(measurement['Epoch'] == epoch)]
    if len(one_epoch.index) > 4:
        try:
            x_wls, v_wls, cov_x, cov_v = wls.WLS_onePosition(one_epoch)  # epoch like period 1s,10s or ....
            ecef_list.append((x_wls, v_wls, cov_x, cov_v))  # unit m, m/s and use tuple to store nhiều variable
            utcTimeMillis_list.append(one_epoch['utcTimeMillis'].iloc[0])  # Lấy giá trị utcTimeMillis đầu tiên cho epoch này
        except Exception as e:
            print(f"Error in WLS_onePosition for epoch {epoch}: {e}")

# Kiểm tra danh sách `utcTimeMillis_list` và `ecef_list`
assert len(utcTimeMillis_list) == len(ecef_list), "The lengths of utcTimeMillis_list and ecef_list do not match"

# Nếu các mảng có kích thước phù hợp, ta có thể stack chúng lại
x_wls_array = np.array([x[0] for x in ecef_list])
if len(x_wls_array) == 0:
    raise ValueError("x_wls_array is empty")

# In ra độ dài của các mảng để kiểm tra
print(f"Length of x_wls_array: {len(x_wls_array)}")
print(f"Length of utcTimeMillis_list: {len(utcTimeMillis_list)}")

# Chuyển đổi từ ECEF sang tọa độ địa lý (latitude, longitude, altitude)
try:
    lla_array = navpy.ecef2lla(x_wls_array.T)
    lla_array = np.array(lla_array).T  # Chuyển đổi thành mảng numpy và chuyển vị
except Exception as e:
    raise ValueError(f"Error in converting ECEF to LLA: {e}")

# In ra độ dài của mảng lla_array để kiểm tra
print(f"Length of lla_array: {len(lla_array)}")

# Kiểm tra kích thước của `lla_array` và `utcTimeMillis_list`
assert len(lla_array) == len(utcTimeMillis_list), "The lengths of lla_array and utcTimeMillis_list do not match"

# Tạo DataFrame và lưu thành file CSV
position_array = [(time, lat, lon) for time, (lat, lon, alt) in zip(utcTimeMillis_list, lla_array)]
lla_df = pd.DataFrame(position_array, columns=['Time', 'Latitude', 'Longitude'])
lla_df.to_csv('calculated_position_wls.csv', index=False)

print(lla_df.head())


: 

In [ ]:
# # Kalman Filter
# # Transistion ecef_list to each array numpy 
# x_wls = np.array([x[0] for x in ecef_list])
# v_wls = np.array([x[1] for x in ecef_list])
# cov_x = np.array([x[2] for x in ecef_list])
# cov_v = np.array([x[3] for x in ecef_list])
# x_kf, P_f = kalman_smoothing(x_wls, v_wls, cov_x, cov_v)
# ecef_list.append(x_kf)
# print(x_kf)  
# Kiểm tra và chuẩn hóa kích thước của các phần tử trong ecef_list
# max_len = max(len(x[0]) for x in ecef_list)

# # Hàm để chuẩn hóa kích thước của một phần tử
# def pad_to_max_len(array, max_len):
#     if len(array) < max_len:
#         pad_width = max_len - len(array)
#         array = np.pad(array, ((0, pad_width), (0, 0)), mode='constant', constant_values=0)
#     return array

# ecef_list_padded = [(pad_to_max_len(x[0], max_len), pad_to_max_len(x[1], max_len), pad_to_max_len(x[2], max_len), pad_to_max_len(x[3], max_len)) for x in ecef_list]

# Chuyển đổi ecef_list thành các mảng numpy riêng lẻ
# x_wls = np.array([x[0] for x in ecef_list_padded])
# v_wls = np.array([x[1] for x in ecef_list_padded])
# cov_x = np.array([x[2] for x in ecef_list_padded])
# cov_v = np.array([x[3] for x in ecef_list_padded])

# x_wls = np.array([x[0] for x in ecef_list])
# v_wls = np.array([x[1] for x in ecef_list])
# cov_x = np.array([x[2] for x in ecef_list])
# cov_v = np.array([x[3] for x in ecef_list])
# # Áp dụng Kalman Filter
# x_kf, P_f = kalman_smoothing(x_wls, v_wls, cov_x, cov_v)
# ecef_list.append(x_kf)
# print(x_kf, P_f)  # In ra giá trị ước lượng vị trí



: 

In [ ]:

# Kiểm tra cấu trúc dữ liệu trong ecef_list và loại bỏ các epoch có kích thước không đồng nhất
filtered_ecef_list = []
filtered_utcTimeMillis_list = []

for i, ecef in enumerate(ecef_list):
    if ecef[2].shape == (3, 3) and ecef[3].shape == (3, 3):
        filtered_ecef_list.append(ecef)
        filtered_utcTimeMillis_list.append(utcTimeMillis_list[i])
    else:
        print(f"Skipping Epoch {i} due to inconsistent shape: cov_x shape = {ecef[2].shape}, cov_v shape = {ecef[3].shape}")

# Nếu các mảng có kích thước phù hợp, ta có thể stack chúng lại
x_wls = np.array([x[0] for x in filtered_ecef_list])
v_wls = np.array([x[1] for x in filtered_ecef_list])
cov_x = np.array([x[2] for x in filtered_ecef_list])
cov_v = np.array([x[3] for x in filtered_ecef_list])

# Áp dụng Kalman Filter
try:
    x_wls, v_wls, cov_x, cov_v = exclude_interpolate_outlier(x_wls, v_wls, cov_x, cov_v)
    x_kf, P_f = kalman_smoothing(x_wls, v_wls, cov_x, cov_v)
    print(x_kf, P_f)  # In ra giá trị ước lượng vị trí

    # Chuyển đổi kết quả Kalman Filter từ ECEF sang tọa độ địa lý
    try:
        lla_kf = navpy.ecef2lla(x_kf.T)
        lla_kf = np.array(lla_kf).T  # Chuyển đổi thành mảng numpy và chuyển vị
    except Exception as e:
        raise ValueError(f"Error in converting ECEF to LLA for Kalman Filter results: {e}")

    # Tạo DataFrame và in ra kết quả Time, Latitude, Longitude sau khi áp dụng Kalman Filter
    kf_position_array = [(time, lat, lon) for time, (lat, lon, alt) in zip(filtered_utcTimeMillis_list, lla_kf)]
    kf_df = pd.DataFrame(kf_position_array, columns=['utcTimeMillis', 'Latitude', 'Longitude'])
    print(kf_df.head())

    # Lưu kết quả sau khi áp dụng Kalman Filter thành file CSV
    kf_df.to_csv('kf_position_data.csv', index=False)
except Exception as e:
    print(f"Error during Kalman Filter application: {e}")


: 